In [31]:
from get_matrix_from_cnn import *
from get_notes_and_image_paths import *
from get_word_embeddings import *
from image_feature_dataset import *
from load_glove_840B_300d import *
from make_study_dictionary import *
import torch
import torch.nn as nn
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from torch.autograd import Variable
import math
import torch.optim as optim
import numpy as np

In [32]:
test_dict = main_get_notes_and_image_paths("test")

In [43]:
test_dict["p10/p10046166/s50051329/427446c1-881f5cce-85191ce1-91a58ba9-0a57d3f5.jpg"]

['<S>',
 'Lateral',
 'view',
 'somewhat',
 'limited',
 'due',
 'to',
 'overlying',
 'motion',
 'artifact',
 '.',
 '<s>',
 'The',
 'lungs',
 'are',
 'low',
 'in',
 'volume',
 '.',
 '<s>',
 'There',
 'is',
 'no',
 'focal',
 'airspace',
 'consolidation',
 'to',
 'suggest',
 'pneumonia',
 '.',
 '<s>',
 'A',
 '1',
 '.',
 '<s>',
 '2-cm',
 'calcified',
 'granuloma',
 'just',
 'below',
 'the',
 'medial',
 'aspect',
 'of',
 'the',
 'right',
 'hemidiaphragm',
 'is',
 'unchanged',
 'from',
 'prior',
 'study',
 '.',
 '<s>',
 'No',
 'pleural',
 'effusions',
 'or',
 'pulmonary',
 'edema',
 '.',
 '<s>',
 'There',
 'is',
 'no',
 'pneumothorax',
 '.',
 '<s>',
 'The',
 'inferior',
 'sternotomy',
 'wire',
 'is',
 'fractured',
 'but',
 'unchanged',
 '.',
 '<s>',
 'Surgical',
 'clips',
 'and',
 'vascular',
 'markers',
 'in',
 'the',
 'thorax',
 'are',
 'related',
 'to',
 'prior',
 'CABG',
 'surgery',
 '.',
 '<s>',
 '</s>']

In [33]:
words = main_get_word_embeddings()

Loading Glove Model
Done. 23094  words loaded!


In [ ]:
example_inputs = torch.randn((10, 16, 1024))

In [39]:
# make word embedding dictionary
weights = np.zeros((len(set(words.keys())), 300))
word2ind = {}
current_ind = 0
for word in list(words.keys()):
    word2ind[word] = current_ind
    weights[current_ind] = words[word]
    current_ind += 1

In [ ]:
def DecoderMask(target_seq, padding_index=word2ind["<S>"]): # target_seq is the clinical note being built by decoder
    target_msk = (target_seq != padding_index).unsqueeze(1)
    size = target_seq.size(1) # get seq_len for matrix
    nopeak_mask = np.triu(np.ones((1, size, size)), k=1).astype('uint8')
    nopeak_mask = Variable(torch.from_numpy(nopeak_mask) == 0)
    target_msk = target_msk & nopeak_mask
    return target_msk.double()

In [30]:
wdim = 300
heads = 8
N = 6
ndim = 1024
weight = weights
model = Transformer(ndim, wdim, N, weight, heads)
for p in model.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)
# this code is very important! It initialises the parameters with a
# range of values that stops the signal fading or getting too big.
optim = torch.optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.98), eps=1e-9)

SyntaxError: invalid syntax (<ipython-input-30-19b32bc473f4>, line 5)

In [ ]:
def train_model(epochs, print_every=100):
    model.train()
    start = time.time()
    temp = start
    total_loss = 0
    for epoch in range(epochs):
        for i, batch in enumerate(train_iter):
            src = # fill in
            trg = # fill in
            # the French sentence we input has all words except
            # the last, as it is using each word to predict the next
            trg_input = trg[:, :-1]
            # the words we are trying to predict
            targets = trg[:, 1:].contiguous().view(-1)
            # create function to make masks using mask code above
            trg_mask = DecoderMask(trg_input, padding_index=)
            preds = model(src, trg_input, trg_mask)
            optim.zero_grad()
            loss = F.cross_entropy(preds.view(-1, preds.size(-1)),
            results, ignore_index=target_pad)
            loss.backward()
            optim.step()
            total_loss += loss.data[0]
            if (i + 1) % print_every == 0:
                loss_avg = total_loss / print_every
                print("time = %dm, epoch %d, iter = %d, loss = %.3f,
                %ds per %d iters" % ((time.time() - start) // 60,
                epoch + 1, i + 1, loss_avg, time.time() - temp,
                print_every))
                total_loss = 0
                temp = time.time()

In [24]:
class Transformer(nn.Module):
    def __init__(self, ndim, wdim, N, weight, heads):
        super().__init__()
        self.encoder = Encoder(ndim, wdim, N, heads)
        self.decoder = Decoder(wdim, N, weight, heads)
        self.out = nn.Linear(wdim, vdim)
    def forward(self, src, trg, trg_mask):
        e_outputs = self.encoder(src)
        d_output = self.decoder(trg, e_outputs, trg_mask)
        output = self.out(d_output)
        return output

In [25]:
class Encoder(nn.Module):
    def __init__(self, ndim, wdim, N, heads, seq_len=16, pdropout=0.1):
        super().__init__()
        self.N = N
        self.embed = LinearEncoder(ndims=ndims, embedding_size=wdim, pdropout=pdropout)
        self.pe = PositionalEncoder(wdim=wdim, dropout=0.1, max_seq_len=seq_len)
        self.layers = get_clones(EncoderLayer(wdim, heads), N)
        self.norm = Norm(wdim)
    def forward(self, src, mask=None):
        x = self.embed(src)
        x = self.pe(x)
        for i in range(N):
            x = self.layers[i](x, mask)
        return self.norm(x)
    
class Decoder(nn.Module):
    def __init__(self, wdim, N, weight, heads, pdropout=0.1):
        super().__init__()
        self.N = N
        self.embed = nn.Embedding.from_pretrained(weight).requires_grad=True
        self.pe = PositionalEncoder(wdim=wdim, dropout=0.1, max_seq_len=64)
        self.layers = get_clones(DecoderLayer(wdim, heads), N)
        self.norm = Norm(wdim)
    def forward(self, trg, e_outputs, mask):
        x = self.embed(trg)
        x = self.pe(x)
        for i in range(self.N):
            x = self.layers[i](x, e_outputs, mask)
        return self.norm(x)

In [26]:
# build a decoder layer with two multi-head attention layers and
# one feed-forward layer
class DecoderLayer(nn.Module):
    def __init__(self, wdim, heads, pdropout=0.1):
        super().__init__()
        self.norm_1 = Norm(wdim)
        self.norm_2 = Norm(wdim)
        self.norm_3 = Norm(wdim)
        
        self.dropout_1 = nn.Dropout(pdropout)
        self.dropout_2 = nn.Dropout(pdropout)
        self.dropout_3 = nn.Dropout(pdropout)
        
        self.attn_1 = MultiHeadAttention(heads, wdim)
        self.attn_2 = MultiHeadAttention(heads, wdim)
        self.ff = FeedForward(wdim)#.cuda()
        
    def forward(self, x, e_outputs, mask):
        x2 = self.norm_1(x)
        x = x + self.dropout_1(self.attn_1(x2, x2, x2, None))
        x2 = self.norm_2(x)
        x = x + self.dropout_2(self.attn_2(x2, e_outputs, e_outputs,
        mask))
        x2 = self.norm_3(x)
        x = x + self.dropout_3(self.ff(x2))
        return x

In [27]:
# build an encoder layer with one multi-head attention layer and one # feed-forward layer
class EncoderLayer(nn.Module):
    def __init__(self, wdim, heads, pdropout = 0.1):
        super().__init__()
        self.norm_1 = Norm(wdim)
        self.norm_2 = Norm(wdim)
        self.attn = MultiHeadAttention(heads, wdim)
        self.ff = FeedForward(wdim)
        self.dropout_1 = nn.Dropout(pdropout)
        self.dropout_2 = nn.Dropout(pdropout)
        
    def forward(self, x, mask=None): # encoder mask set to None
        x2 = self.norm_1(x)
        x = x + self.dropout_1(self.attn(x2,x2,x2, mask))
        x2 = self.norm_2(x)
        x = x + self.dropout_2(self.ff(x2))
        return x

In [28]:
class Norm(nn.Module):
    def __init__(self, wdim, eps = 1e-6):
        super().__init__()
    
        self.size = wdim
        # create two learnable parameters to calibrate normalisation
        self.alpha = nn.Parameter(torch.ones(self.size))
        self.bias = nn.Parameter(torch.zeros(self.size))
        self.eps = eps
    def forward(self, x):
        norm = self.alpha * (x - x.mean(dim=-1, keepdim=True)) / (x.std(dim=-1, keepdim=True) + self.eps) + self.bias
        return norm

In [29]:
class FeedForward(nn.Module):
    def __init__(self, wdim, d_ff=2048, pdropout = 0.1):
        super().__init__() 
        # We set d_ff as a default to 2048
        self.linear_1 = nn.Linear(wdim, d_ff)
        self.dropout = nn.Dropout(pdropout)
        self.linear_2 = nn.Linear(d_ff, wdim)
    def forward(self, x):
        x = self.dropout(F.relu(self.linear_1(x)))
        x = self.linear_2(x)
        return x

In [19]:
class MultiHeadAttention(nn.Module):
    def __init__(self, heads, wdim, pdropout = 0.1): # note that heads must divide wdim with 0 remainder
        super().__init__()
        
        self.wdim = wdim
        self.d_k = wdim // heads
        self.h = heads
        self.q_linear = nn.Linear(wdim, wdim)
        self.v_linear = nn.Linear(wdim, wdim)
        self.k_linear = nn.Linear(wdim, wdim)
        self.dropout = nn.Dropout(pdropout)
        self.out = nn.Linear(wdim, wdim)
        
    def attention(q, k, v, d_k, mask=None, dropout=None):
    
        scores = torch.matmul(q, k.transpose(-2, -1)) /  math.sqrt(d_k)

        if mask is not None:
            mask = mask.unsqueeze(1)
            scores = scores.masked_fill(mask == 0, -1e9)
        scores = F.softmax(scores, dim=-1)
    
        if dropout is not None:
            scores = dropout(scores)
        
        output = torch.matmul(scores, v)
        return output
    
    def forward(self, q, k, v, mask=None):
        bs = q.size(0)
        # perform linear operation and split into h heads
        k = self.k_linear(k).view(bs, -1, self.h, self.d_k)
        q = self.q_linear(q).view(bs, -1, self.h, self.d_k)
        v = self.v_linear(v).view(bs, -1, self.h, self.d_k)
        # transpose to get dimensions bs * h * sl * wdim
        k = k.transpose(1,2)
        q = q.transpose(1,2)
        v = v.transpose(1,2)
	# calculate attention using function we will define next
        scores = attention(q, k, v, self.d_k, mask, self.dropout)
        # concatenate heads and put through final linear layer
        concat = scores.transpose(1,2).contiguous().view(bs, -1, self.wdim)
        output = self.out(concat)
        return output

In [20]:
def DecoderMask(target_seq, padding_index): # target_seq is the clinical note being built by decoder
    target_msk = (target_seq != padding_index).unsqueeze(1)
    size = target_seq.size(1) # get seq_len for matrix
    nopeak_mask = np.triu(np.ones((1, size, size)), k=1).astype('uint8')
    nopeak_mask = Variable(torch.from_numpy(nopeak_mask) == 0)

    target_msk = target_msk & nopeak_mask
    return target_msk.double()

In [21]:
class LinearEncoder(nn.Module):
    def __init__(self, ndims=1024, embedding_size=300, pdropout=0.3):
        super(LinearEncoder, self).__init__()
        self.pdropout = pdropout
        self.ndims = ndims
        self.embedding_size = embedding_size
        
        # Layers
        self.fc1 = nn.Linear(ndims, embedding_size)
        self.dropout = nn.Dropout(pdropout)
        
    def forward(self, x):
        """Inputs of forward function
        Args:
            x: the sequence fed to the linear encoder model (required)
        Shape of x: (batch size, seq_len, ndims)
            output: (batch size, seq_len, embedding_size)

        """
        x = self.fc1(x)
        x = self.dropout(x)
        
        return x

In [22]:
class PositionalEncoder(nn.Module):
    def __init__(self, wdim, max_seq_len=64):
        super().__init__()
        self.wdim = wdim
        self.max_seq_len = max_seq_len
        
        # create constant 'pe' matrix with values dependant on 
        # pos and i
        pe = torch.zeros(max_seq_len, wdim)
        for pos in range(max_seq_len):
            for i in range(0, wdim, 2):
                pe[pos, i] = math.sin(pos / (10000 ** ((i)/wdim)))
                pe[pos, i + 1] = math.cos(pos / (10000 ** ((i)/wdim)))
                
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
 
    
    def forward(self, x):
        """Inputs of forward function
        Args:
            x: the sequence out of the linear encoder (required)
        Shape of x: (batch size, sequence length, embedding_size)
            output: (batch size, sequence length, embedding_size) # with positional encodings
        """
        # make embeddings relatively larger
        x = x * math.sqrt(self.wdim)
        #add constant to embedding
        seq_len = x.size(1)
        x += Variable(self.pe[:,:seq_len], requires_grad=False)
        return x